In [ ]:
# python imports
from math import ceil
from datetime import datetime

# third-party
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read & Prep Dataset

In [ ]:
vulns = pd.read_csv('../output/vulns-labelled.csv', low_memory=False)

In [ ]:
vulns.info()

In [ ]:
vulns['cve_published_date'] =\
    pd.to_datetime(vulns['cve_published_date'], format='%Y-%m-%d', errors='coerce')

vulns['exploit_published_date'] =\
    pd.to_datetime(vulns['exploit_published_date'], format='%Y-%m-%d', errors='coerce')

In [ ]:
vulns['exploitable'] = vulns['exploit_count'].apply(lambda value: 1 if value > 0 else 0)

In [ ]:
conditions = [
    ((vulns['base_score'] <= 3.9 )),
    ((vulns['base_score'] >= 4.0) & (vulns['base_score'] <= 6.9)),
    ((vulns['base_score'] >= 7.0) & (vulns['base_score'] <= 8.9)),
    ((vulns['base_score'] >= 9.0))    
]

choices = ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL']

vulns['base_severity'] = np.select(conditions, choices, default='OTHER')
vulns['base_severity'] = pd.Categorical(vulns.base_severity, categories=choices, ordered=True)

In [ ]:
# transforming columns in lists
vulns['part'] = vulns['part'].apply(eval)
vulns['vendor'] = vulns['vendor'].apply(eval)

# Utils

In [ ]:
def to_1D(series):
     return pd.Series([x for _list in series for x in _list])

## Vulns year distribution

In [ ]:
plotdata = vulns.groupby(vulns['cve_published_date'].dt.year, sort=True)['cve_id'].count()
plotdata.plot(kind='bar', title='Vulnerabilities by Year', xlabel='', ylabel='', rot=45)

## Part distribution

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

part_counts = to_1D(vulns['part']).value_counts().values
parts = [value / sum(part_counts) * 100 for value in part_counts]

ax.bar(['aplication', 'operating\n system', 'hardware'], parts)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Part Distribution", size=14)

plt.xticks(rotation=45)
plt.show()

## Vendor distribution

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

vendor_counts = to_1D(vulns['vendor']).value_counts()[1:11]
vendors = [value / sum(vendor_counts) * 100 for value in vendor_counts]

ax.bar(vendor_counts.index, vendors)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Vendor Distribution", size=14)

plt.xticks(rotation=45)
plt.show()

## CIA distribution

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12, 4))

ci_counts = vulns['confidentiality_impact'].value_counts().values
cis = [value / sum(ci_counts) * 100 for value in ci_counts]

ax1.bar(['HIGH', 'NONE', 'LOW'], cis)
ax1.set_ylabel("Proportion (%)", size = 12)
ax1.set_title("Confidentiality Distribution", size=14)

ii_counts = vulns['integrity_impact'].value_counts().values
iis = [value / sum(ii_counts) * 100 for value in ii_counts]

ax2.bar(['HIGH', 'NONE', 'LOW'], iis)
ax2.set_ylabel("Proportion (%)", size = 12)
ax2.set_title("Integrity Distribution", size=14)

ai_counts = vulns['availability_impact'].value_counts().values
ais = [value / sum(ai_counts) * 100 for value in ai_counts]

ax3.bar(['HIGH', 'NONE', 'LOW'], ais)
ax3.set_ylabel("Proportion (%)", size = 12)
ax3.set_title("Availability Distribution", size=14)

plt.show()

## CVSS distribution

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

value_counts = vulns['base_severity'].value_counts().sort_index()
values = [ value / sum(value_counts.values) for value in value_counts.values]

colors = ['green', 'yellow', 'orange', 'red']

ax.bar(value_counts.index, values, color=colors)
ax.set_ylabel('Percentage (%)')
ax.set_xlabel('Labels')

plt.show()

## Mitre and OWASP lists

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 4))

mitre_counts = vulns['mitre_top_25'].value_counts().values
mitres = [value / sum(mitre_counts) * 100 for value in mitre_counts]

ax1.bar(['In Mitre', 'Not in Mitre'], mitres)
ax1.set_ylabel("Proportion (%)", size = 12)
ax1.set_title("Mitre top 25 Distribution", size=14)

owasp_counts = vulns['owasp_top_10'].value_counts().values
owasps = [value / sum(owasp_counts) * 100 for value in owasp_counts]

ax2.bar(['In OWASP', 'Not in OWASP'], owasps)
ax2.set_ylabel("Proportion (%)", size = 12)
ax2.set_title("OWASP top 10 Distribution", size=14)

plt.show()

## Exploits per Year

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

exploit_counts = vulns.groupby(vulns['exploit_published_date'].dt.year, sort=True)['cve_id'].count()
exploits = [value / sum(exploit_counts.values) * 100 for value in exploit_counts.values]

ax.bar(exploit_counts.index, exploits)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Exploits Distribution by Year", size=14)

plt.show()

## Exploitable Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

values_array = vulns['exploitable'].value_counts().values
values = [value / sum(values_array) * 100 for value in values_array]

ax.bar(['Not exploitable', 'Exploitable'], values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Exploitable Distribution", size=14)

plt.show()

## Updatable

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

values_array = vulns['update_available'].value_counts().values
values = [value / sum(values_array) * 100 for value in values_array]

ax.bar(['Update unavailable', 'Update available'], values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Updatable CVEs Distribution", size=14)

plt.show()

## EPSS

In [ ]:
columns = [
    'cve_id', 'cve_published_date', 'base_score', 
    'confidentiality_impact', 'integrity_impact', 
    'availability_impact', 'exploit_count', 'epss']
vulns.sort_values(by='epss', ascending=False)[columns].head(10)

## Attack type distribution

In [ ]:
fig, ax = plt.subplots(figsize = (14,4))

attacks = vulns.loc[~vulns['attack_type'].isnull()]
attacks['attack_type'] = attacks['attack_type'].apply(eval)

attack_counts = to_1D(attacks['attack_type']).value_counts()[:10]
attacks = [round(value / sum(attack_counts) * 100) for value in attack_counts]

ax.bar(attack_counts.index, attacks)

ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Attack Types Distribution", size=14)

plt.xticks(rotation=45)
plt.show()

## Audience

In [ ]:
audience = vulns.loc[~vulns['audience'].isnull()].sort_values(by='audience', ascending=False)
audience.head(10)[['cve_id', 'base_score', 'exploit_count', 'audience']]

## Google Trends

In [ ]:
trends = vulns.loc[~vulns['google_interest'].isnull()].sort_values(by='google_interest', ascending=False)
trends.head(10)[['cve_id', 'base_score', 'exploit_count', 'google_trend', 'google_interest', 'epss']]

## Topology

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['topology'].value_counts()
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Topology Distribution", size=14)

plt.show()

## Type

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['asset_type'].value_counts()
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Type Distribution", size=14)

plt.show()

## Environment

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['environment'].value_counts()
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Environment Distribution", size=14)

plt.show()

## Data

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['sensitive_data'].value_counts()
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Data Distribution", size=14)

plt.show()

## End-of-Life

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['end_of_life'].value_counts().rename(index={0: 'False', 1: 'True'})
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("End-of-Life Distribution", size=14)

plt.show()

## Honeypot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

value_counts = vulns['honeypot'].value_counts().rename(index={0: 'False', 1: 'True'})
values = [value / sum(values_array) * 100 for value in value_counts.values]

ax.bar(value_counts.index, values)
ax.set_ylabel("Proportion (%)", size = 12)
ax.set_title("Honeypot Distribution", size=14)

plt.show()

## Labelling Test

In [ ]:
total = vulns.shape[0]

In [ ]:
critical = vulns.loc[
    (vulns['base_score'] >= 7.0) &
    (
        (vulns['confidentiality_impact'] == 'HIGH') & 
        (vulns['integrity_impact'] == 'HIGH') & 
        (vulns['availability_impact'] == 'HIGH')
    ) &
    (
        (vulns['mitre_top_25'] == 1) | (vulns['owasp_top_10'] == 1)
    ) &
    (
        (vulns['exploit_count'] > 0) | (~vulns['audience'].isnull()) | (vulns['epss'] >= 0.9)
    )
]

vulns = vulns.loc[~vulns['cve_id'].isin(critical['cve_id'])]

print(f'critical correspond to {critical.shape[0] / total:.3f}% do total, which are {critical.shape[0]} vulns.')

In [ ]:
moderate = vulns.loc[
    (vulns['base_score'] >= 7.0) &
    (
        ((vulns['confidentiality_impact'] == 'HIGH') | (vulns['confidentiality_impact'] == 'LOW')) & 
        ((vulns['integrity_impact'] == 'HIGH') | (vulns['integrity_impact'] == 'LOW')) &
        ((vulns['availability_impact'] == 'HIGH') | (vulns['availability_impact'] == 'LOW'))
    ) &
    (
        (vulns['mitre_top_25'] == 1) | (vulns['owasp_top_10'] == 1)
    )
]

vulns = vulns.loc[~vulns['cve_id'].isin(moderate['cve_id'])]

print(f'moderate correspond to {moderate.shape[0] / total:.3f}% do total, which are {moderate.shape[0]} vulns.')

In [ ]:
important = vulns

print(f'important correspond to {important.shape[0] / total:.3f}% do total, which ar {important.shape[0]} vulns.')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

values = [important.shape[0], moderate.shape[0], critical.shape[0]]
values = [ value / total for value in values ]

labels = ['MODERATE', 'IMPORTANT', 'CRITICAL']

colors = ['yellow', 'orange', 'red']

ax.bar(labels, values, color=colors)
ax.set_ylabel('Percentage (%)')
ax.set_xlabel('Labels')

plt.show()